# $(4,8)$--Matriods

In this notebook, we provide the computations used in the proof of Proposition 5.3, which asserts that $\operatorname{Gr}(\mathsf{Q};\mathbb{C})$ is smooth for all $\mathbb{C}$--realizable $(4,8)$--matroids. 

In [27]:
using Oscar
using Combinatorics
pm = Polymake



#pull simple 4-8 matroids from polymake databases
db = Polymake.Polydb.get_db();
collection = db["Matroids.Small"];
d4n8 = Polymake.Polydb.find(collection, Dict("RANK" => 4, "SIMPLE"=>true, "N_ELEMENTS"=>8));

In [2]:
cd("..")

In [7]:
currentDir = pwd() # make sure you are running from the main directory.
include(joinpath(currentDir, "src/fileHandling.jl"))
include(joinpath(currentDir, "src/matroid_realization.jl"))
include(joinpath(currentDir, "src/Jacobian_Criterion.jl"))
include(joinpath(currentDir, "src/reduceTSC.jl"))
include(joinpath(currentDir, "src/isolate3Lines.jl"))

to_revlex (generic function with 1 method)

First we isolate those that satisfy the three lines property, i.e., every element of the ground set is contained in at least 4 planes (rank 3 flat with at least 4 elements). There are **92** such matroids.

In [28]:
#counts number of planes an element of the ground set is in
function in_planes(n,L)

    Ln = [l for l in L if n in l]

    return(length(Ln))

end



d4C8 = subsets(collect(1:8), 4);
d4C8 = sort(d4C8, by =  x-> reverse(x));
#isolate simple connected matroids satisfying 4 lines property
planes_4 = []

for t in d4n8
    
   # println(t)

    Qt = Matroid(t)

    L = [h for h in hyperplanes(Qt) if length(h)>3]

    ns = [n for n in 1:8 if in_planes(n,L)>3]

    if (length(ns) == 8&&is_connected(Qt))
                push!(planes_4,to_revlex(Qt,d4C8))
    end
end

This data is precomputed in `d4n8/data/four_planes_connected_4_8.dat`.

In [29]:
planes_4_precomputed = vec(readlines(joinpath(currentDir, "d4n8/data/4planes_connected_4_8.dat")));

In [30]:
length(planes_4_precomputed)

92

In [31]:
Set(planes_4) == Set(planes_4_precomputed)

true

A pair of matroid and circuit is recorded as a `*/0` vector. The first 8 entries record the circuit (e.g., `***0*00*` is the circuit `[1,2,3,5,8]`) and the rest record the matroid in the usual revlex basis encoding. There is a unique matroid in `d4n8/data/four_planes_connected_4_8.dat` that does not admit a $5$ element circuit, which we study at the end of this notebook.

The file `d4n8/data/nonrealizable_4_8.dat` contains those matroids in `d4n8/four_planes_connected_4_8.dat` that are not realizable over $\mathbb{Q}$. There are **25** such matroids.

In [21]:
nonrealizable = vec(readlines(joinpath(currentDir, "d4n8/nonrealizable_4_8.dat")))
length(nonrealizable)

25

In [35]:
nonrealizable = vec(readlines(joinpath(currentDir, "d4n8/data/nonrealizable_4_8.dat")))

test_nonrealizable = []

for Qstr in nonrealizable
    Q = matroid_from_revlex_basis_encoding(Qstr[9:78], 4, 8)
    As = rank_plus1_circuits(Q) 
    MR = new_matroid_realization_space(Q, As[1]; F=QQ, saturate=true)
    push!(test_nonrealizable, !MR.representable)
end

In [34]:
all(test_nonrealizable)

true

The files `zero_ideal_4_8.dat` and `univariate_ideal_4_8.dat`, in `d4n8/data/` record realizable matroids and a maximal circuit.

**Note:** One could check that these are smooth using the `singular_locus` function in `src/Jacobian_Criterion.jl`. To make this computation more transparent, we carry out the smoothness check explicitly.

The precomputed data `d4n8/zero_ideal_4_8.dat` contains the matroids in `d4n8/4planes_connected_4_8.dat` such that the vanishing ideal of their realization space reduces to the trivial ideal. There are **63** such matroids. These realization spaces of these matroids are of course smooth.

In [36]:
zero_ideal = vec(readlines(joinpath(currentDir, "d4n8/data/zero_ideal_4_8.dat")))
length(zero_ideal)

63

In [39]:
test_zero = []
for cir_Qstr in zero_ideal   
    Qstr = cir_Qstr[9:78]
    A = [i for i in 1:8 if string(cir_Qstr[i]) == "*"] 
    Q = matroid_from_revlex_basis_encoding(Qstr, 4, 8) 
    MR = new_matroid_realization_space(Q, A; F=QQ, saturate=true)
        
    MR = reduce_ideal_full(MR)
    I = MR.defining_ideal
    push!(test_zero, iszero(I))
end

In [40]:
all(test_zero)

true

The precomputed data `d4n8/data/univariate_ideal_4_8.dat` contains the matroids in `d4n8/data/4planes_connected_4_8.dat` for which there exists some reference circuit such that the vanishing ideal is univariate and therefore principal. There are **3** such matroids.

In [41]:
principal_univariate = vec(readlines(joinpath(currentDir,"d4n8/data/univariate_ideal_4_8.dat")))

3-element Vector{String}:
 "***00**00********0****0*****0**0*****************0****0***********0*******0***"
 "***00**00********0****0*****0**0*****************0****0********0************0*"
 "**0*00**000000*********0************0***0**0*************0***0******00********"

In [42]:
#verify univariate
test_univariate = []

for cir_Qstr in principal_univariate
    A = [i for i in 1:8 if string(cir_Qstr[i]) == "*"] 
    Qstr = cir_Qstr[9:78]
    Q = matroid_from_revlex_basis_encoding(Qstr, 4, 8) 
    MR = new_matroid_realization_space(Q, A; F=QQ, saturate=true)
    
    MR = reduce_ideal_full(MR)
    I = MR.defining_ideal
    length_vs = length(ideal_vars(gens(I)))
    push!(test_univariate, isone(length_vs))
end


In [43]:
all(test_univariate)

true

We now check if the realization spaces of those matroids contained in `d4n8/univariate_ideal_4_8.dat` are smooth.  By the Jacobian criterion, the affine scheme $\mathsf{Spec}(S^{-1}\mathbb{Q}[x_1,\ldots,x_n] / \langle f \rangle)$ is smooth if and only if the ideal

$$
J = \langle f, \frac{\partial f}{\partial x_1}, \ldots, \frac{\partial f}{\partial x_n}\rangle
$$

is the unit ideal in $S^{-1} \mathbb{Q}[x_1,\ldots,x_n]$. This is equivalent to the saturation $(J:S^{\infty})$ being the unit ideal in $\mathbb{Q}[x_1,\ldots,x_n]$. We perform this explicit check here. 

In [44]:

test_smooth_principal = []
for cir_Qstr in principal_univariate
    
    A = [i for i in 1:8 if string(cir_Qstr[i]) == "*"] 
    Qstr = cir_Qstr[9:78]
    Q = matroid_from_revlex_basis_encoding(Qstr, 4, 8) 
    MR = new_matroid_realization_space(Q, A; F=QQ, saturate=true)
        
    MR = reduce_ideal_full(MR)
    R = MR.ambient_ring
    x = gens(R)
    I = MR.defining_ideal
    Igens = gens(I)
    length(Igens) != 1 && error("not principal") 
    JM = jacobian_matrix(R, x, Igens)
    nr, nc = size(JM) 
    J = I + ideal(R, [JM[1,c] for c in 1:nc])
    Sing = stepwise_saturation(J, MR.inequations)
    #Sing = realization_space_2_singular_locus(MR)
    push!(test_smooth_principal, isone(Sing))
end

In [45]:
all(test_smooth_principal)

true

We conclude by examining the unique simple, connected, $(4,8)$-matroid satisfying the four planes property that does not have a reference circuit.

In [55]:
no_circ = []
for i in planes_4
    Qi = matroid_from_revlex_basis_encoding(i,4,8)
    Cs = rank_plus1_circuits(Qi)
    if length(Cs) == 0
        push!(no_circ,i)
    end
end
no_circ

1-element Vector{Any}:
 "0********0****0*****0**0***00************00***0**0*****0****0********0"

This data is precomputed in `d4n8/data/no_ref_circ_4_8.dat`

In [56]:
no_circs_precomputed = vec(readlines(joinpath(currentDir,"d4n8/data/no_ref_circ_4_8.dat")))

1-element Vector{String}:
 "0********0****0*****0**0***00************00***0**0*****0****0********0"

In [57]:
no_circs_precomputed == no_circ

true

We study this matroid via its thin Schubert cell.

In [58]:
M = matroid_from_revlex_basis_encoding(no_circs[1],4,8)
T = matroid_to_reduced_TSC_min_basis(M,QQ)

(QQMPolyRingElem[1], QQMPolyRingElem[x_{3, 1}, x_{2, 4}, x_{1, 3}, x_{4, 4}, x_{4, 2}, x_{4, 3}, x_{3, 4}], [1, 2, 3, 5])

As the defining ideal is the unit ideal, this matroid is not realizable over $\mathbb{Q}$.

Finally, we show that our sorting of the matroids in `d4n8/4_8_four_planes_connected.dat` was exhaustive.

In [59]:
all_matroids_with_circ = Set(union(principal_univariate,zero_ideal,nonrealizable))
all_no_circs = union([cir_Qstr[9:78] for cir_Qstr in all_matroids_with_circ],no_circs)


Set(all_no_circs) == Set(planes_4_precomputed)

true